# Experiment 007: Eazy Optimizer Validation

Validate the output from the Eazy C++ optimizer.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def check_overlaps_for_n(df, n):
    """Check if configuration N has overlapping trees. Returns (has_overlap, overlap_area)"""
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return True, float('inf')
    
    polygons = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        polygons.append(create_tree_polygon(x, y, deg))
    
    # Check for overlaps
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-10:
                    return True, intersection.area
    return False, 0

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    return max(all_points.max(axis=0) - all_points.min(axis=0))

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    total = 0
    for n in range(1, 201):
        total += compute_score_for_n(df, n)
    return total

print("Functions defined")

Functions defined


In [2]:
# Load and validate the Eazy optimizer output
df_eazy = pd.read_csv('/home/code/experiments/007_eazy_optimizer/submission.csv')
print(f"Loaded {len(df_eazy)} rows")
print(df_eazy.head())

Loaded 20100 rows
      id                        x                       y  \
0  001_0  s-31.763345364712542107  s43.851215526625153984   
1  002_0    s0.154097069621355831  s-0.038540742694794905   
2  002_1   s-0.154097069621373012  s-0.561459257305223947   
3  003_0    s1.136371522500167819   s0.783701416888897162   
4  003_1    s1.234055695842160016   s1.275999500663759001   

                       deg  
0   s45.000000000000135003  
1  s203.629377730656869971  
2   s23.629377730656791812  
3  s113.563586170568299849  
4   s66.370622269343002131  


In [3]:
# Check for overlaps in all N values
print("Checking for overlaps...")
overlap_count = 0
for n in range(1, 201):
    has_overlap, _ = check_overlaps_for_n(df_eazy, n)
    if has_overlap:
        overlap_count += 1
        if overlap_count <= 10:
            print(f"  N={n}: OVERLAP DETECTED")

print(f"\nTotal N values with overlaps: {overlap_count}/200")

Checking for overlaps...



Total N values with overlaps: 0/200


In [4]:
# Compute total score
if overlap_count == 0:
    eazy_score = compute_total_score(df_eazy)
    print(f"Eazy optimizer score: {eazy_score:.6f}")
    
    # Compare with baseline
    df_baseline = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
    baseline_score = compute_total_score(df_baseline)
    print(f"Baseline score: {baseline_score:.6f}")
    print(f"Improvement: {baseline_score - eazy_score:.9f}")
else:
    print("Cannot compute score - overlaps detected")

Eazy optimizer score: 70.659944


Baseline score: 70.659959
Improvement: 0.000015000


In [5]:
# Save as submission if valid
if overlap_count == 0:
    import shutil
    shutil.copy('/home/code/experiments/007_eazy_optimizer/submission.csv', '/home/submission/submission.csv')
    print("Saved to /home/submission/submission.csv")
    
    # Verify
    df_verify = pd.read_csv('/home/submission/submission.csv')
    verify_score = compute_total_score(df_verify)
    print(f"Verified submission score: {verify_score:.6f}")

Saved to /home/submission/submission.csv


Verified submission score: 70.659944


In [6]:
# Summary
print("="*60)
print("EXPERIMENT 007 SUMMARY: Eazy Optimizer")
print("="*60)
print(f"Baseline score: {baseline_score:.6f}")
print(f"Eazy optimizer score: {eazy_score:.6f}")
print(f"Improvement: {baseline_score - eazy_score:.9f}")
print(f"Overlaps: {overlap_count}/200")
print("="*60)

EXPERIMENT 007 SUMMARY: Eazy Optimizer
Baseline score: 70.659959
Eazy optimizer score: 70.659944
Improvement: 0.000015000
Overlaps: 0/200
